In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [8]:
## Import Pre-Processed Data

import pickle

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("labels.pickle","rb")
labels = pickle.load(pickle_in)

In [9]:
def Model1(input_shape = (200, 200, 1), classes = Y_test.shape[1]):
    """"
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((2, 2))(X_input)

    
    # Stage 1
    X = Conv2D(32, (7,7), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    
    # Stage 2
    X = Conv2D(32, (4,4), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    
    # Stage 3
    X = Conv2D(32, (2,2), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    
    # output layer
    X = Flatten()(X)
#    X = Dense(classes, activation='relu')(X)
#    X = Dropout(0.1)(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='Model1')

    return model

In [10]:
# Normalize image vectors
X_train = X_train/255
X_test = X_test/255

In [11]:
model = Model1(input_shape = (200, 200, 1), classes = Y_test.shape[1])

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
200/200 [==============================] - 61s 307ms/step - loss: 8.9039 - acc: 0.4150
Epoch 2/10
200/200 [==============================] - 56s 280ms/step - loss: 8.7244 - acc: 0.4500
Epoch 3/10
200/200 [==============================] - 56s 278ms/step - loss: 8.5151 - acc: 0.4450
Epoch 4/10
200/200 [==============================] - 55s 277ms/step - loss: 8.4271 - acc: 0.4500
Epoch 5/10
200/200 [==============================] - 55s 277ms/step - loss: 8.3027 - acc: 0.4650
Epoch 6/10
200/200 [==============================] - 56s 282ms/step - loss: 8.1858 - acc: 0.4850
Epoch 7/10
200/200 [==============================] - 56s 279ms/step - loss: 8.1907 - acc: 0.4850
Epoch 8/10
200/200 [==============================] - 56s 278ms/step - loss: 8.1433 - acc: 0.4950
Epoch 9/10
200/200 [==============================] - 56s 279ms/step - loss: 8.1397 - acc: 0.4950
Epoch 10/10
200/200 [==============================] - 55s 276ms/step 

In [14]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

50/50 [==============================] - 2s 47ms/step
Loss = 9.993343048095703
Test Accuracy = 0.38


In [15]:
model.save('basic_model')